In [162]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import (Imputer, PolynomialFeatures, StandardScaler)
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction import DictVectorizer
import numpy as np
from sklearn.metrics import roc_auc_score

In [24]:


df_train = pd.read_csv("C:/Users/ThomasRowe/Datalytyx/Delivery - Documents/Data Science/Data/Santander_Value_Prediction/train.csv")


In [43]:
df = pd.read_csv("C:/Users/ThomasRowe/Python37-32/adult.csv")

In [148]:
from sklearn.datasets import make_regression

features, labels = make_regression(10000, 200, effective_rank = 50)

features = pd.DataFrame(features)

In [72]:

labels = pd.DataFrame(df['Income'])



features = df.loc[:, [col for col in df.columns if (col != 'Income')]]
features.index = df['Age']

In [48]:
features.head()

,Age,Edu,Num,Edu.1,Edunum,Marital,Job,Relationship,Race,Gender,capgain,caploss,Hours,Country,Income
Age,,,,,,,,,,,,,,,
39,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
50,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
38,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
53,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
28,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [49]:
import scipy as sp

In [163]:
class CorrelationSelector(TransformerMixin, BaseEstimator):
    
    
    def __init__(self, n_columns = 50):
        
        self.n_columns = n_columns
   

    def fit(self, X, y = None):
        
        if not isinstance(X, pd.DataFrame):
            raise TypeError('X must be a pandas DataFrame')
        
        #Find correlations
        correlations_target = []

        for i in X.columns:
            
            corr, p_test = sp.stats.spearmanr(X[i], y)
            correlations_target.append(corr)
            
        corr_train_target = pd.Series(correlations_target, index = X.columns).abs()
        
        #Check for nans
        if corr_train_target.isna().sum() != 0: 
            raise ValueError('Feature corelations contain nans from {}'.format(self.__class__))
            
        self.largest = corr_train_target.nlargest(self.n_columns)
        
        #Print a metric
        self.correlation_total = corr_train_target.sum()
        self.correlation_score = self.largest.sum()
        
        self.correlation_percentage = self.correlation_score/self.correlation_total*100
        
        print('The {} features with largest correlation to the labels contain {:.2f}% of the total correlation score.'
              .format(self.n_columns, self.correlation_percentage))
        
        
        self.most_correlated_cols = self.largest.index
            
        return self
                 
    
    def transform(self, X):
        
        
        return X.loc[:, self.most_correlated_cols] 

In [164]:
corr_test = CorrelationSelector(50)
corr_test.fit(features,labels)

The 50 features with largest correlation to the labels contain 57.34% of the total correlation score.


CorrelationSelector(n_columns=50)

In [165]:
corr_test.transform(features)

,60,136,122,172,106,92,154,124,187,151,...,174,20,169,28,199,1,183,148,155,176
0,0.006405,0.003707,0.005734,-0.000963,0.004632,-0.002759,0.008454,0.003097,-0.001327,0.005973,...,-0.007110,-0.001561,-0.003360,-0.004950,-0.003120,-0.006525,0.000087,-0.002264,2.127047e-03,-0.000783
1,-0.000901,0.001260,0.006746,-0.003836,0.006921,0.005161,0.000770,-0.018625,-0.000869,-0.011364,...,0.005064,0.000948,-0.003401,-0.001348,0.008756,0.003116,-0.005641,0.009844,1.095141e-03,0.007235
2,0.005079,0.013309,-0.002589,0.006453,0.002300,-0.003526,-0.000929,0.000818,0.009241,0.001461,...,-0.001721,-0.002631,0.007222,-0.009949,-0.006146,0.009738,-0.001149,0.000041,2.082923e-03,-0.005093
3,-0.008375,-0.000189,0.003412,-0.003558,-0.002422,0.005815,-0.009762,0.003332,-0.003082,-0.000673,...,0.002628,-0.007336,0.000225,0.004935,0.000891,0.007988,0.000058,0.000596,9.488145e-04,0.002161
4,-0.001964,-0.004150,0.006754,0.005190,0.005871,-0.008792,-0.014556,0.005025,-0.000604,0.012312,...,-0.002435,0.000689,-0.006100,-0.006050,-0.004162,-0.009520,0.006179,0.001591,-1.233027e-02,0.009865
5,-0.000698,0.003540,-0.000440,0.008091,0.003749,0.002163,-0.007166,0.002904,-0.001309,0.007467,...,0.006538,-0.002351,-0.000246,-0.001032,0.003360,0.000700,-0.010418,0.004125,5.489864e-03,0.007474
6,-0.000732,-0.005410,0.004327,-0.005979,-0.002644,0.002354,-0.009610,0.001214,-0.001203,0.001915,...,0.001595,-0.006739,-0.001650,-0.001308,-0.004311,-0.004504,0.001826,-0.002704,4.091822e-03,0.011488
7,0.009919,0.000884,-0.007618,0.002652,0.002931,0.000458,-0.000937,0.006926,-0.007912,0.006327,...,-0.004752,-0.000421,-0.000991,0.001462,-0.002361,0.003831,0.005581,0.002607,-5.364892e-03,0.002812
8,-0.001053,-0.000807,-0.001208,0.001668,-0.000346,0.003202,-0.012190,0.000064,0.000066,-0.002661,...,0.001058,-0.006510,-0.008706,0.003816,-0.014199,-0.009178,-0.002827,0.006790,-4.835850e-03,0.002885
9,-0.002162,-0.001330,0.003116,0.012963,-0.007027,0.009784,0.003412,0.008221,0.005767,-0.003371,...,0.003777,0.001816,0.004820,-0.009094,0.015739,-0.002625,-0.006194,-0.000326,-3.562032e-03,-0.007446
